In [42]:
import numpy as np
import pandas as pd
import math

In [43]:
# Come from 
# https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources//Data/preprocessed/Cell_line_RMA_proc_basalExp.txt.zip
data = pd.read_csv("data/CCLE_expression.csv",index_col=0)

In [44]:
data.shape

(1389, 19177)

In [45]:
data.head()

,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),NFYA (4800),...,ARHGAP11B (89839),AC004593.2 (1124),AC090517.4 (54816),AL160269.1 (11046),ABCF2-H2BE1 (114483834),POLR2J3 (548644),H2BE1 (114483833),AL445238.1 (647264),GET1-SH3BGR (106865373),AC113348.1 (102724657)
ACH-001113,4.990501,0.000000,7.273702,2.765535,4.480265,0.028569,1.269033,3.058316,6.483171,5.053980,...,1.214125,0.000000,0.111031,0.150560,1.427606,5.781884,0.0,0.000000,0.799087,0.000000
ACH-001289,5.209843,0.545968,7.070604,2.538538,3.510962,0.000000,0.176323,3.836934,4.200850,3.832890,...,1.835924,0.000000,0.310340,0.000000,0.807355,4.704319,0.0,0.000000,0.464668,0.070389
ACH-001339,3.779260,0.000000,7.346425,2.339137,4.254745,0.056584,1.339137,6.724241,3.671293,3.775051,...,1.823749,0.084064,0.176323,0.042644,1.384050,4.931683,0.0,0.028569,0.263034,0.000000
ACH-001538,5.726831,0.000000,7.086189,2.543496,3.102658,0.000000,5.914565,6.099716,4.475733,4.294253,...,0.871844,0.137504,0.263034,2.485427,0.713696,3.858976,0.0,0.000000,0.000000,0.000000
ACH-000242,7.465648,0.000000,6.435462,2.414136,3.864929,0.831877,7.198003,5.452530,7.112492,4.710944,...,2.324811,0.163499,0.163499,0.000000,1.117695,4.990501,0.0,0.000000,0.000000,0.000000


In [46]:
meta = pd.read_csv("data/sample_info.csv",index_col=0)

In [47]:
meta_cosmic = meta.loc[:,"COSMICID"].dropna(inplace=False).astype({'COSMICID': 'int32'})

In [48]:
meta_cosmic.head()

DepMap_ID
ACH-000001    905933
ACH-000002    905938
ACH-000004    907053
ACH-000006    908148
ACH-000007    907795
Name: COSMICID, dtype: int32

In [49]:
meta.head()

,cell_line_name,stripped_cell_line_name,CCLE_Name,alias,COSMICID,sex,source,Achilles_n_replicates,cell_line_NNMD,culture_type,...,primary_or_metastasis,primary_disease,Subtype,age,Sanger_Model_ID,depmap_public_comments,lineage,lineage_subtype,lineage_sub_subtype,lineage_molecular_subtype
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,NIH:OVCAR-3,NIHOVCAR3,NIHOVCAR3_OVARY,OVCAR3,905933.0,Female,ATCC,NaN,NaN,NaN,...,Metastasis,Ovarian Cancer,"Adenocarcinoma, high grade serous",60.0,SIDM00105,NaN,ovary,ovary_adenocarcinoma,high_grade_serous,NaN
ACH-000002,HL-60,HL60,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,905938.0,Female,ATCC,NaN,NaN,NaN,...,Primary,Leukemia,"Acute Myelogenous Leukemia (AML), M3 (Promyelo...",35.0,SIDM00829,NaN,blood,AML,M3,NaN
ACH-000003,CACO2,CACO2,CACO2_LARGE_INTESTINE,"CACO2, CaCo-2",NaN,Male,ATCC,NaN,NaN,NaN,...,NaN,Colon/Colorectal Cancer,Adenocarcinoma,NaN,SIDM00891,NaN,colorectal,colorectal_adenocarcinoma,NaN,NaN
ACH-000004,HEL,HEL,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,907053.0,Male,DSMZ,2.0,-3.092311,Suspension,...,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",30.0,SIDM00594,NaN,blood,AML,M6,NaN
ACH-000005,HEL 92.1.7,HEL9217,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Male,ATCC,2.0,-2.412923,Suspension,...,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",30.0,SIDM00593,NaN,blood,AML,M6,NaN


In [50]:
# Load ccle
dose = pd.read_csv("data/secondary-screen-dose-response-curve-parameters.csv",index_col=0)

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
dose.head()

,depmap_id,ccle_name,screen_id,upper_limit,lower_limit,slope,r2,auc,ec50,ic50,name,moa,target,disease.area,indication,smiles,phase,passed_str_profiling,row_name
broad_id,,,,,,,,,,,,,,,,,,,
BRD-K71847383-001-12-5,ACH-000879,MFE296_ENDOMETRIUM,HTS002,1,2.122352,-0.022826,-0.026964,1.677789,8.415093e+06,NaN,cytarabine,ribonucleotide reductase inhibitor,"POLA1, POLB, POLD1, POLE",hematologic malignancy,"acute lymphoblastic leukemia (ALL), chronic ly...",Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)...,Launched,True,ACH-000879
BRD-K71847383-001-12-5,ACH-000320,PSN1_PANCREAS,HTS002,1,1.325174,-0.237504,-0.147274,1.240300,9.643742e+00,NaN,cytarabine,ribonucleotide reductase inhibitor,"POLA1, POLB, POLD1, POLE",hematologic malignancy,"acute lymphoblastic leukemia (ALL), chronic ly...",Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)...,Launched,True,ACH-000320
BRD-K71847383-001-12-5,ACH-001145,OC316_OVARY,HTS002,1,2.089350,-0.302937,0.193893,1.472333,2.776687e-02,NaN,cytarabine,ribonucleotide reductase inhibitor,"POLA1, POLB, POLD1, POLE",hematologic malignancy,"acute lymphoblastic leukemia (ALL), chronic ly...",Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)...,Launched,True,ACH-001145
BRD-K71847383-001-12-5,ACH-000873,KYSE270_OESOPHAGUS,HTS002,1,1.311820,-0.209393,-0.005460,1.207160,2.654701e+00,NaN,cytarabine,ribonucleotide reductase inhibitor,"POLA1, POLB, POLD1, POLE",hematologic malignancy,"acute lymphoblastic leukemia (ALL), chronic ly...",Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)...,Launched,True,ACH-000873
BRD-K71847383-001-12-5,ACH-000855,KYSE150_OESOPHAGUS,HTS002,1,1.369799,-0.277530,0.132818,1.229332,5.889041e-01,NaN,cytarabine,ribonucleotide reductase inhibitor,"POLA1, POLB, POLD1, POLE",hematologic malignancy,"acute lymphoblastic leukemia (ALL), chronic ly...",Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)...,Launched,True,ACH-000855


In [52]:
dose_tab = pd.crosstab(index=dose['depmap_id'], columns = dose['name'],values=dose['auc'],aggfunc='mean')

In [53]:
dose_tab.head()

name,1-azakenpaullone,1-naphthyl-PP1,1-phenylbiguanide,10-deacetylbaccatin,10-hydroxycamptothecin,12-O-tetradecanoylphorbol-13-acetate,2'-MeCCPA,"2,3-DCPE",2-methoxyestradiol,3'-fluorobenzylspiperone,...,warfarin,xanomeline,xilobam,xylazine,zaldaride,zaleplon,zardaverine,ziprasidone,zolantidine,zolmitriptan
depmap_id,,,,,,,,,,,,,,,,,,,,,
ACH-000007,1.200721,0.906230,0.840409,0.861057,0.236149,0.678549,NaN,0.977291,0.886259,0.945707,...,NaN,0.849267,0.796757,1.475364,0.880131,NaN,0.832131,0.942748,NaN,1.380169
ACH-000008,0.895310,0.864177,0.986722,0.860633,0.347057,1.042324,1.264433,1.103182,0.896848,1.212847,...,1.166609,0.842375,0.828385,1.134751,0.929585,1.140489,0.888494,0.945813,NaN,0.931826
ACH-000011,0.821426,0.812357,0.894664,0.928655,0.455415,0.551885,1.377388,0.961209,0.907681,0.989196,...,1.211876,0.861407,0.839902,1.358932,0.941054,NaN,0.834375,0.955548,NaN,NaN
ACH-000012,0.874960,0.814528,0.946367,0.816810,0.302450,0.826459,1.381436,0.906660,0.811619,0.894749,...,1.298038,0.837195,0.845846,1.257923,0.916330,0.951544,0.816745,0.923753,0.937082,1.314640
ACH-000013,0.917994,0.860783,0.954582,0.826188,0.087541,0.752544,1.115056,0.842884,0.867457,1.199869,...,1.588719,0.783282,0.815430,1.492371,0.940541,0.893297,0.797681,0.953010,0.960252,0.925308


In [54]:
# Select the index that have dose infomation
dose_idx = dose_tab.index

In [55]:
dose_idx

Index(['ACH-000007', 'ACH-000008', 'ACH-000011', 'ACH-000012', 'ACH-000013',
       'ACH-000014', 'ACH-000015', 'ACH-000018', 'ACH-000019', 'ACH-000021',
       ...
       'ACH-001075', 'ACH-001113', 'ACH-001128', 'ACH-001145', 'ACH-001190',
       'ACH-001239', 'ACH-001306', 'ACH-001307', 'ACH-001318', 'ACH-001321'],
      dtype='object', name='depmap_id', length=480)

In [56]:
# Select the index that overlapping the meta data
intersect_idx = dose_tab.index.intersection(meta_cosmic.index)

In [57]:
ach2cosmic = meta_cosmic[intersect_idx]

In [58]:
ach2cosmic.index

Index(['ACH-000007', 'ACH-000008', 'ACH-000012', 'ACH-000015', 'ACH-000018',
       'ACH-000019', 'ACH-000021', 'ACH-000023', 'ACH-000030', 'ACH-000035',
       ...
       'ACH-000987', 'ACH-000988', 'ACH-000991', 'ACH-000997', 'ACH-001016',
       'ACH-001024', 'ACH-001075', 'ACH-001190', 'ACH-001306', 'ACH-001307'],
      dtype='object', length=348)

In [59]:
# drugs = ["I-BET-762","ibrutinib","dabrafenib","trametinib","cisplatin","tamoxifen",#"JQ1"
#          "entinostat","gefitinib"]

In [60]:
#select_drug_dose = dose_tab.loc[:,drugs]
select_drug_dose = dose_tab
select_drug_dose = select_drug_dose.reset_index()
select_drug_dose.head()

name,depmap_id,1-azakenpaullone,1-naphthyl-PP1,1-phenylbiguanide,10-deacetylbaccatin,10-hydroxycamptothecin,12-O-tetradecanoylphorbol-13-acetate,2'-MeCCPA,"2,3-DCPE",2-methoxyestradiol,...,warfarin,xanomeline,xilobam,xylazine,zaldaride,zaleplon,zardaverine,ziprasidone,zolantidine,zolmitriptan
0,ACH-000007,1.200721,0.906230,0.840409,0.861057,0.236149,0.678549,NaN,0.977291,0.886259,...,NaN,0.849267,0.796757,1.475364,0.880131,NaN,0.832131,0.942748,NaN,1.380169
1,ACH-000008,0.895310,0.864177,0.986722,0.860633,0.347057,1.042324,1.264433,1.103182,0.896848,...,1.166609,0.842375,0.828385,1.134751,0.929585,1.140489,0.888494,0.945813,NaN,0.931826
2,ACH-000011,0.821426,0.812357,0.894664,0.928655,0.455415,0.551885,1.377388,0.961209,0.907681,...,1.211876,0.861407,0.839902,1.358932,0.941054,NaN,0.834375,0.955548,NaN,NaN
3,ACH-000012,0.874960,0.814528,0.946367,0.816810,0.302450,0.826459,1.381436,0.906660,0.811619,...,1.298038,0.837195,0.845846,1.257923,0.916330,0.951544,0.816745,0.923753,0.937082,1.314640
4,ACH-000013,0.917994,0.860783,0.954582,0.826188,0.087541,0.752544,1.115056,0.842884,0.867457,...,1.588719,0.783282,0.815430,1.492371,0.940541,0.893297,0.797681,0.953010,0.960252,0.925308


In [61]:
select_drug_dose = select_drug_dose.set_index("depmap_id")

In [62]:
cosmic_drug_dose = select_drug_dose.loc[ach2cosmic.index,:]
cosmic_drug_dose.head()

name,1-azakenpaullone,1-naphthyl-PP1,1-phenylbiguanide,10-deacetylbaccatin,10-hydroxycamptothecin,12-O-tetradecanoylphorbol-13-acetate,2'-MeCCPA,"2,3-DCPE",2-methoxyestradiol,3'-fluorobenzylspiperone,...,warfarin,xanomeline,xilobam,xylazine,zaldaride,zaleplon,zardaverine,ziprasidone,zolantidine,zolmitriptan
ACH-000007,1.200721,0.906230,0.840409,0.861057,0.236149,0.678549,NaN,0.977291,0.886259,0.945707,...,NaN,0.849267,0.796757,1.475364,0.880131,NaN,0.832131,0.942748,NaN,1.380169
ACH-000008,0.895310,0.864177,0.986722,0.860633,0.347057,1.042324,1.264433,1.103182,0.896848,1.212847,...,1.166609,0.842375,0.828385,1.134751,0.929585,1.140489,0.888494,0.945813,NaN,0.931826
ACH-000012,0.874960,0.814528,0.946367,0.816810,0.302450,0.826459,1.381436,0.906660,0.811619,0.894749,...,1.298038,0.837195,0.845846,1.257923,0.916330,0.951544,0.816745,0.923753,0.937082,1.314640
ACH-000015,0.997159,0.895351,NaN,0.712753,0.171208,0.968407,1.604969,1.107416,0.778368,1.075989,...,1.266804,0.809983,1.220899,NaN,1.202025,1.455858,0.776302,0.831692,2.044948,NaN
ACH-000018,0.877318,0.878532,0.904814,1.327718,0.282880,1.044834,0.947369,0.969656,0.950678,0.937633,...,NaN,1.000001,1.111303,1.093489,0.914858,0.930743,0.832751,1.382313,NaN,0.913824


In [63]:
cosmic_drug_dose["COSMIC_ID"] = ach2cosmic.values

In [64]:
ach2cosmic.head()

ACH-000007     907795
ACH-000008     910921
ACH-000012    1240146
ACH-000015     908471
ACH-000018     724812
Name: COSMICID, dtype: int32

In [65]:
cosmic_drug_dose = cosmic_drug_dose.set_index("COSMIC_ID")
cosmic_drug_dose.head()

name,1-azakenpaullone,1-naphthyl-PP1,1-phenylbiguanide,10-deacetylbaccatin,10-hydroxycamptothecin,12-O-tetradecanoylphorbol-13-acetate,2'-MeCCPA,"2,3-DCPE",2-methoxyestradiol,3'-fluorobenzylspiperone,...,warfarin,xanomeline,xilobam,xylazine,zaldaride,zaleplon,zardaverine,ziprasidone,zolantidine,zolmitriptan
COSMIC_ID,,,,,,,,,,,,,,,,,,,,,
907795,1.200721,0.906230,0.840409,0.861057,0.236149,0.678549,NaN,0.977291,0.886259,0.945707,...,NaN,0.849267,0.796757,1.475364,0.880131,NaN,0.832131,0.942748,NaN,1.380169
910921,0.895310,0.864177,0.986722,0.860633,0.347057,1.042324,1.264433,1.103182,0.896848,1.212847,...,1.166609,0.842375,0.828385,1.134751,0.929585,1.140489,0.888494,0.945813,NaN,0.931826
1240146,0.874960,0.814528,0.946367,0.816810,0.302450,0.826459,1.381436,0.906660,0.811619,0.894749,...,1.298038,0.837195,0.845846,1.257923,0.916330,0.951544,0.816745,0.923753,0.937082,1.314640
908471,0.997159,0.895351,NaN,0.712753,0.171208,0.968407,1.604969,1.107416,0.778368,1.075989,...,1.266804,0.809983,1.220899,NaN,1.202025,1.455858,0.776302,0.831692,2.044948,NaN
724812,0.877318,0.878532,0.904814,1.327718,0.282880,1.044834,0.947369,0.969656,0.950678,0.937633,...,NaN,1.000001,1.111303,1.093489,0.914858,0.930743,0.832751,1.382313,NaN,0.913824


In [66]:
# Comes from 
#ftp://ftp.sanger.ac.uk/pub/project/cancerrxgene/releases/current_release/GDSC2_fitted_dose_response_25Feb20.xlsx
# gdsc = pd.read_csv("data/Cell_line_RMA_proc_basalExp.txt",sep="\t")
label = pd.read_excel("data/GDSC2_fitted_dose_response_25Feb20.xlsx")


In [67]:
GDSC_id = list(set(label.COSMIC_ID))

In [68]:
label_info = label.loc[:,["COSMIC_ID","CELL_LINE_NAME","AUC","DRUG_NAME"]]

In [69]:
label_info.head()

,COSMIC_ID,CELL_LINE_NAME,AUC,DRUG_NAME
0,749709,HCC1954,0.975203,Camptothecin
1,749710,HCC1143,0.978464,Camptothecin
2,749711,HCC1187,0.995100,Camptothecin
3,749712,HCC1395,0.867832,Camptothecin
4,749713,HCC1599,0.777532,Camptothecin


In [70]:
Cap_drugs = ["I-BET-762","Ibrutinib","Dabrafenib","Trametinib","Cisplatin","Tamoxifen",#"JQ1"
         "Entinostat","Gefitinib"]
# selected_drug_sens = label_info.loc[label_info.DRUG_NAME in drugs,:]

In [71]:
#selected_index = [d in drugs for d in label_info.DRUG_NAME.values]

In [72]:
#seleced_labels = label_info[selected_index]

In [73]:
selected_alldrug_sens = pd.crosstab(index=label_info['COSMIC_ID'], columns = label_info['DRUG_NAME'],values=label_info['AUC'],aggfunc='mean')

In [74]:
#gdsc_tab = selected_alldrug_sens.loc[:,Cap_drugs]
gdsc_tab = selected_alldrug_sens
gdsc_tab.head()

DRUG_NAME,5-Fluorouracil,ABT737,AGI-5198,AGI-6780,AMG-319,AT13148,AZ6102,AZ960,AZD1208,AZD1332,...,Vorinostat,WEHI-539,WIKI4,WZ4003,Wee1 Inhibitor,Wnt-C59,XAV939,YK-4-279,ZM447439,Zoledronate
COSMIC_ID,,,,,,,,,,,,,,,,,,,,,
683665,0.988746,0.491973,0.991472,0.921248,0.990548,0.977377,0.855110,0.670322,0.993164,0.993162,...,0.744778,0.306364,0.841007,0.945011,0.713712,0.852600,0.939151,0.695062,NaN,0.991812
683667,0.976741,0.655735,0.972966,0.972888,0.971282,0.964687,0.837857,0.860038,0.964506,0.971642,...,0.695245,0.922247,0.948386,0.882791,0.827897,0.961412,0.942191,0.784064,NaN,0.932254
684052,0.949240,0.646030,0.972733,0.958843,0.988567,0.938209,0.794876,0.807361,0.985132,0.919386,...,0.800694,0.949760,0.949546,0.955502,0.837360,0.969871,0.973265,0.617951,NaN,0.986251
684057,0.959395,NaN,NaN,0.948520,NaN,NaN,NaN,0.730258,0.952070,0.908745,...,NaN,NaN,NaN,0.825403,NaN,0.965422,0.973363,NaN,NaN,0.983723
684059,0.858744,0.785829,0.966310,0.963419,0.955631,0.952941,0.848514,0.756592,0.985749,0.964014,...,0.785805,0.948819,0.944875,0.915169,0.881386,0.961724,0.985809,0.670564,NaN,0.817600


In [75]:
# Change GDSC ot upper cases
gdsc_drug_upper = [ d.upper() for d in gdsc_tab.columns.values]

gdsc_tab.columns = gdsc_drug_upper
gdsc_tab.head()

,5-FLUOROURACIL,ABT737,AGI-5198,AGI-6780,AMG-319,AT13148,AZ6102,AZ960,AZD1208,AZD1332,...,VORINOSTAT,WEHI-539,WIKI4,WZ4003,WEE1 INHIBITOR,WNT-C59,XAV939,YK-4-279,ZM447439,ZOLEDRONATE
COSMIC_ID,,,,,,,,,,,,,,,,,,,,,
683665,0.988746,0.491973,0.991472,0.921248,0.990548,0.977377,0.855110,0.670322,0.993164,0.993162,...,0.744778,0.306364,0.841007,0.945011,0.713712,0.852600,0.939151,0.695062,NaN,0.991812
683667,0.976741,0.655735,0.972966,0.972888,0.971282,0.964687,0.837857,0.860038,0.964506,0.971642,...,0.695245,0.922247,0.948386,0.882791,0.827897,0.961412,0.942191,0.784064,NaN,0.932254
684052,0.949240,0.646030,0.972733,0.958843,0.988567,0.938209,0.794876,0.807361,0.985132,0.919386,...,0.800694,0.949760,0.949546,0.955502,0.837360,0.969871,0.973265,0.617951,NaN,0.986251
684057,0.959395,NaN,NaN,0.948520,NaN,NaN,NaN,0.730258,0.952070,0.908745,...,NaN,NaN,NaN,0.825403,NaN,0.965422,0.973363,NaN,NaN,0.983723
684059,0.858744,0.785829,0.966310,0.963419,0.955631,0.952941,0.848514,0.756592,0.985749,0.964014,...,0.785805,0.948819,0.944875,0.915169,0.881386,0.961724,0.985809,0.670564,NaN,0.817600


In [76]:
# Change CCLE ot upper cases
ccle_drug_upper = [ d.upper() for d in dose_tab.columns.values]

dose_tab.columns = ccle_drug_upper
dose_tab.head()

,1-AZAKENPAULLONE,1-NAPHTHYL-PP1,1-PHENYLBIGUANIDE,10-DEACETYLBACCATIN,10-HYDROXYCAMPTOTHECIN,12-O-TETRADECANOYLPHORBOL-13-ACETATE,2'-MECCPA,"2,3-DCPE",2-METHOXYESTRADIOL,3'-FLUOROBENZYLSPIPERONE,...,WARFARIN,XANOMELINE,XILOBAM,XYLAZINE,ZALDARIDE,ZALEPLON,ZARDAVERINE,ZIPRASIDONE,ZOLANTIDINE,ZOLMITRIPTAN
depmap_id,,,,,,,,,,,,,,,,,,,,,
ACH-000007,1.200721,0.906230,0.840409,0.861057,0.236149,0.678549,NaN,0.977291,0.886259,0.945707,...,NaN,0.849267,0.796757,1.475364,0.880131,NaN,0.832131,0.942748,NaN,1.380169
ACH-000008,0.895310,0.864177,0.986722,0.860633,0.347057,1.042324,1.264433,1.103182,0.896848,1.212847,...,1.166609,0.842375,0.828385,1.134751,0.929585,1.140489,0.888494,0.945813,NaN,0.931826
ACH-000011,0.821426,0.812357,0.894664,0.928655,0.455415,0.551885,1.377388,0.961209,0.907681,0.989196,...,1.211876,0.861407,0.839902,1.358932,0.941054,NaN,0.834375,0.955548,NaN,NaN
ACH-000012,0.874960,0.814528,0.946367,0.816810,0.302450,0.826459,1.381436,0.906660,0.811619,0.894749,...,1.298038,0.837195,0.845846,1.257923,0.916330,0.951544,0.816745,0.923753,0.937082,1.314640
ACH-000013,0.917994,0.860783,0.954582,0.826188,0.087541,0.752544,1.115056,0.842884,0.867457,1.199869,...,1.588719,0.783282,0.815430,1.492371,0.940541,0.893297,0.797681,0.953010,0.960252,0.925308


In [77]:
# Select common drugs between two databases
commen_drug_columns = dose_tab.columns.intersection(gdsc_tab.columns)

In [78]:
exteneded_drug_columns = dose_tab.columns.union(gdsc_tab.columns)

In [79]:
exteneded_drug_columns

Index(['1-AZAKENPAULLONE', '1-NAPHTHYL-PP1', '1-PHENYLBIGUANIDE',
       '10-DEACETYLBACCATIN', '10-HYDROXYCAMPTOTHECIN',
       '12-O-TETRADECANOYLPHORBOL-13-ACETATE', '2'-MECCPA', '2,3-DCPE',
       '2-METHOXYESTRADIOL', '3'-FLUOROBENZYLSPIPERONE',
       ...
       'ZK-93426', 'ZK811752', 'ZLN005', 'ZM-306416', 'ZM-447439', 'ZM447439',
       'ZOLANTIDINE', 'ZOLEDRONATE', 'ZOLMITRIPTAN', 'ZSTK-474'],
      dtype='object', length=1557)

In [80]:
commen_drug_columns

Index(['5-FLUOROURACIL', 'AZ960', 'AZD2014', 'AZD4547', 'AZD5363', 'AZD5438',
       'AZD6482', 'AZD7762', 'AZD8055', 'BI-2536', 'BIBR-1532', 'BMS-345541',
       'BMS-754807', 'GSK1904529A', 'I-BET-762', 'LY2109761', 'MG-132',
       'MK-1775', 'MK-2206', 'OSI-027', 'OTX015', 'P22077', 'PLX-4720',
       'PRT062607', 'VE-822', 'AFATINIB', 'ALISERTIB', 'ALPELISIB', 'AXITINIB',
       'BORTEZOMIB', 'BUPARLISIB', 'CAMPTOTHECIN', 'CARMUSTINE', 'CEDIRANIB',
       'CISPLATIN', 'CRIZOTINIB', 'CYCLOPHOSPHAMIDE', 'CYTARABINE',
       'DABRAFENIB', 'DASATINIB', 'DINACICLIB', 'DOCETAXEL', 'ENTINOSTAT',
       'EPIRUBICIN', 'ERLOTINIB', 'FLUDARABINE', 'FORETINIB', 'FULVESTRANT',
       'GEFITINIB', 'GEMCITABINE', 'IBRUTINIB', 'IRINOTECAN', 'LAPATINIB',
       'LINSITINIB', 'MITOXANTRONE', 'NAVITOCLAX', 'NELARABINE', 'NILOTINIB',
       'NIRAPARIB', 'OLAPARIB', 'OSIMERTINIB', 'OXALIPLATIN', 'PACLITAXEL',
       'PALBOCICLIB', 'PEVONEDISTAT', 'RIBOCICLIB', 'RUXOLITINIB',
       'SELUMETINIB', 'SOR

In [107]:
gdsc_ids = [ "DATA."+str(i) for i in gdsc_tab.index.values]

In [109]:
gdsc_tab.index = gdsc_ids

In [112]:
gdsc_tab

,5-FLUOROURACIL,ABT737,AGI-5198,AGI-6780,AMG-319,AT13148,AZ6102,AZ960,AZD1208,AZD1332,...,VORINOSTAT,WEHI-539,WIKI4,WZ4003,WEE1 INHIBITOR,WNT-C59,XAV939,YK-4-279,ZM447439,ZOLEDRONATE
DATA.683665,0.988746,0.491973,0.991472,0.921248,0.990548,0.977377,0.855110,0.670322,0.993164,0.993162,...,0.744778,0.306364,0.841007,0.945011,0.713712,0.852600,0.939151,0.695062,NaN,0.991812
DATA.683667,0.976741,0.655735,0.972966,0.972888,0.971282,0.964687,0.837857,0.860038,0.964506,0.971642,...,0.695245,0.922247,0.948386,0.882791,0.827897,0.961412,0.942191,0.784064,NaN,0.932254
DATA.684052,0.949240,0.646030,0.972733,0.958843,0.988567,0.938209,0.794876,0.807361,0.985132,0.919386,...,0.800694,0.949760,0.949546,0.955502,0.837360,0.969871,0.973265,0.617951,NaN,0.986251
DATA.684057,0.959395,NaN,NaN,0.948520,NaN,NaN,NaN,0.730258,0.952070,0.908745,...,NaN,NaN,NaN,0.825403,NaN,0.965422,0.973363,NaN,NaN,0.983723
DATA.684059,0.858744,0.785829,0.966310,0.963419,0.955631,0.952941,0.848514,0.756592,0.985749,0.964014,...,0.785805,0.948819,0.944875,0.915169,0.881386,0.961724,0.985809,0.670564,NaN,0.817600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DATA.1660034,0.832363,0.918031,0.955663,0.976561,NaN,0.842167,0.925704,0.951805,0.975662,0.960958,...,0.882738,0.955368,0.881543,0.975864,0.918233,0.954496,0.949847,0.757741,NaN,0.979268
DATA.1660035,0.963426,0.938137,0.949747,0.941915,NaN,NaN,NaN,0.863267,0.985205,0.968613,...,0.784177,0.977847,0.934910,0.963458,0.971475,0.955843,0.968306,0.875885,NaN,0.981690
DATA.1660036,0.924514,NaN,NaN,0.983145,NaN,NaN,NaN,0.957422,0.993269,0.993249,...,NaN,NaN,NaN,0.987716,NaN,0.971806,0.993065,NaN,NaN,0.992914
DATA.1674021,0.831864,0.914564,0.971497,0.971378,0.968631,0.966074,0.848079,0.838834,0.980687,0.944238,...,0.747011,0.949813,0.905999,0.943055,0.758515,0.979652,0.959474,0.760292,NaN,0.983978


In [114]:
GDSC_label_common = gdsc_tab.loc[:,commen_drug_columns]
GDSC_label_common.head()

,5-FLUOROURACIL,AZ960,AZD2014,AZD4547,AZD5363,AZD5438,AZD6482,AZD7762,AZD8055,BI-2536,...,TENIPOSIDE,TOPOTECAN,TOZASERTIB,TRAMETINIB,UPROSERTIB,VENETOCLAX,VINBLASTINE,VINCRISTINE,VINORELBINE,VORINOSTAT
DATA.683665,0.988746,0.670322,0.950175,0.968525,0.961817,0.764785,NaN,0.492478,NaN,NaN,...,0.427355,0.618331,NaN,0.994088,0.936213,0.933957,0.483393,0.072019,0.645842,0.744778
DATA.683667,0.976741,0.860038,0.847139,0.994262,0.876508,0.812361,NaN,0.648063,NaN,NaN,...,0.851304,0.861507,NaN,0.989145,0.848736,0.887319,0.599965,0.291710,0.710986,0.695245
DATA.684052,0.949240,0.807361,0.778575,0.948738,0.935697,0.713411,NaN,0.589910,NaN,NaN,...,0.574182,0.466748,NaN,0.965742,0.953161,0.971618,0.460939,0.174685,0.640397,0.800694
DATA.684057,0.959395,0.730258,0.674690,NaN,NaN,0.844561,NaN,NaN,NaN,NaN,...,0.747489,0.834007,NaN,NaN,0.777959,NaN,NaN,0.240994,NaN,NaN
DATA.684059,0.858744,0.756592,0.750252,0.982099,0.934719,0.761498,NaN,0.475531,NaN,NaN,...,0.467434,0.312764,NaN,0.989165,0.920961,0.964455,0.498375,0.106963,0.529711,0.785805


In [115]:
# gdsc_ids = [ "DATA."+str(i) for i in GDSC_label_common.index.values]
# GDSC_label_common.index = gdsc_ids
GDSC_label_common.head()

,5-FLUOROURACIL,AZ960,AZD2014,AZD4547,AZD5363,AZD5438,AZD6482,AZD7762,AZD8055,BI-2536,...,TENIPOSIDE,TOPOTECAN,TOZASERTIB,TRAMETINIB,UPROSERTIB,VENETOCLAX,VINBLASTINE,VINCRISTINE,VINORELBINE,VORINOSTAT
DATA.683665,0.988746,0.670322,0.950175,0.968525,0.961817,0.764785,NaN,0.492478,NaN,NaN,...,0.427355,0.618331,NaN,0.994088,0.936213,0.933957,0.483393,0.072019,0.645842,0.744778
DATA.683667,0.976741,0.860038,0.847139,0.994262,0.876508,0.812361,NaN,0.648063,NaN,NaN,...,0.851304,0.861507,NaN,0.989145,0.848736,0.887319,0.599965,0.291710,0.710986,0.695245
DATA.684052,0.949240,0.807361,0.778575,0.948738,0.935697,0.713411,NaN,0.589910,NaN,NaN,...,0.574182,0.466748,NaN,0.965742,0.953161,0.971618,0.460939,0.174685,0.640397,0.800694
DATA.684057,0.959395,0.730258,0.674690,NaN,NaN,0.844561,NaN,NaN,NaN,NaN,...,0.747489,0.834007,NaN,NaN,0.777959,NaN,NaN,0.240994,NaN,NaN
DATA.684059,0.858744,0.756592,0.750252,0.982099,0.934719,0.761498,NaN,0.475531,NaN,NaN,...,0.467434,0.312764,NaN,0.989165,0.920961,0.964455,0.498375,0.106963,0.529711,0.785805


In [83]:
CCLE_label_common = dose_tab.loc[:,commen_drug_columns]
CCLE_label_common.head()

,5-FLUOROURACIL,AZ960,AZD2014,AZD4547,AZD5363,AZD5438,AZD6482,AZD7762,AZD8055,BI-2536,...,TENIPOSIDE,TOPOTECAN,TOZASERTIB,TRAMETINIB,UPROSERTIB,VENETOCLAX,VINBLASTINE,VINCRISTINE,VINORELBINE,VORINOSTAT
depmap_id,,,,,,,,,,,,,,,,,,,,,
ACH-000007,0.650901,0.803749,0.913730,NaN,0.873329,0.730186,0.845277,0.581497,0.774935,0.576837,...,0.587615,0.479828,0.723243,0.337852,0.828375,1.257363,0.499241,0.631508,0.810523,0.879708
ACH-000008,0.904002,0.810568,0.767467,NaN,0.890474,0.815721,0.727160,0.582944,0.615447,0.576850,...,0.577777,0.590030,0.868619,0.578623,0.754824,0.937708,0.600889,0.433690,0.618706,0.914406
ACH-000011,0.899686,0.774886,0.762004,1.490741,0.931512,0.812180,0.777673,0.584851,0.602401,0.561480,...,0.498873,0.676890,0.771770,0.897670,0.895684,0.817704,0.468774,0.653337,0.758796,0.872112
ACH-000012,0.860095,0.737418,0.820056,NaN,0.885907,0.831101,0.763760,0.462838,0.597943,0.444844,...,0.540391,0.503783,0.781231,0.903596,0.883709,1.192620,0.531134,0.433863,0.591041,0.761647
ACH-000013,NaN,0.812942,0.824296,1.628982,1.197040,0.824372,0.680928,0.419215,0.603507,0.422891,...,0.477397,0.289131,0.658417,1.165718,0.786182,1.169608,0.407725,0.154534,0.500894,0.877884


In [118]:
dose_tab

,1-AZAKENPAULLONE,1-NAPHTHYL-PP1,1-PHENYLBIGUANIDE,10-DEACETYLBACCATIN,10-HYDROXYCAMPTOTHECIN,12-O-TETRADECANOYLPHORBOL-13-ACETATE,2'-MECCPA,"2,3-DCPE",2-METHOXYESTRADIOL,3'-FLUOROBENZYLSPIPERONE,...,WARFARIN,XANOMELINE,XILOBAM,XYLAZINE,ZALDARIDE,ZALEPLON,ZARDAVERINE,ZIPRASIDONE,ZOLANTIDINE,ZOLMITRIPTAN
depmap_id,,,,,,,,,,,,,,,,,,,,,
ACH-000007,1.200721,0.906230,0.840409,0.861057,0.236149,0.678549,NaN,0.977291,0.886259,0.945707,...,NaN,0.849267,0.796757,1.475364,0.880131,NaN,0.832131,0.942748,NaN,1.380169
ACH-000008,0.895310,0.864177,0.986722,0.860633,0.347057,1.042324,1.264433,1.103182,0.896848,1.212847,...,1.166609,0.842375,0.828385,1.134751,0.929585,1.140489,0.888494,0.945813,NaN,0.931826
ACH-000011,0.821426,0.812357,0.894664,0.928655,0.455415,0.551885,1.377388,0.961209,0.907681,0.989196,...,1.211876,0.861407,0.839902,1.358932,0.941054,NaN,0.834375,0.955548,NaN,NaN
ACH-000012,0.874960,0.814528,0.946367,0.816810,0.302450,0.826459,1.381436,0.906660,0.811619,0.894749,...,1.298038,0.837195,0.845846,1.257923,0.916330,0.951544,0.816745,0.923753,0.937082,1.314640
ACH-000013,0.917994,0.860783,0.954582,0.826188,0.087541,0.752544,1.115056,0.842884,0.867457,1.199869,...,1.588719,0.783282,0.815430,1.492371,0.940541,0.893297,0.797681,0.953010,0.960252,0.925308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-001239,NaN,0.883204,0.870239,1.013601,0.318023,NaN,NaN,0.870609,0.791788,0.872277,...,0.856010,0.889074,0.892768,1.160533,0.876945,0.862995,0.999955,0.866009,0.942972,0.829851
ACH-001306,0.894745,0.875309,NaN,1.149388,0.288112,1.000935,1.204651,1.183884,0.875710,1.231729,...,0.969187,0.771182,0.867433,1.116206,0.942135,1.169739,0.718992,1.523193,NaN,1.405732
ACH-001307,1.168534,0.853086,1.193130,0.830150,0.141646,1.199518,1.262466,0.962323,0.805852,1.066176,...,0.935440,0.865896,0.906832,1.079176,0.935193,0.957796,0.787185,1.167778,0.931503,1.062078


In [84]:
GDSC2_df = pd.read_csv("data/GDSC2_expression.csv",index_col=0)

In [121]:
GDSC2_df.head()

,TSPAN6,TNMD,DPM1,SCYL3,C1ORF112,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC00514,OR1D5,ZNF234,MYH4,LINC00526,PPY2,KRT18P55,POLRMTP1,UBL5P2,TBC1D3P5
DATA.683665,3.238273,2.982254,10.235491,4.856061,4.078870,9.116236,3.658590,6.145475,5.042464,5.438402,...,3.103752,3.724013,3.981948,2.823245,5.866047,3.095716,3.274367,3.056214,9.446305,3.530871
DATA.683667,7.780713,2.753253,9.960137,4.351073,3.716740,3.222277,8.221606,3.823474,4.756228,5.805642,...,3.175476,3.779354,4.504481,2.690651,3.347520,3.230713,3.102091,3.169188,9.810430,3.266915
DATA.684052,7.301344,2.890533,9.922489,4.125088,3.678987,3.096576,3.588391,4.809305,4.951782,5.089165,...,3.299300,3.762301,4.177345,2.499803,5.054260,3.003521,3.068187,3.135479,9.073222,3.098364
DATA.684057,8.233101,2.824687,10.015884,4.749715,3.839433,3.142754,5.329830,3.272124,5.538055,6.428482,...,3.081750,3.226083,4.666295,2.491254,6.261573,3.031862,3.322455,2.813440,8.893197,3.266184
DATA.684059,8.333466,3.966757,9.793991,3.976923,3.505669,3.079943,3.373640,4.199048,5.794734,5.902391,...,3.055770,3.738082,4.705392,2.667882,3.885425,2.993918,3.219777,3.033869,8.691401,3.279230


In [85]:
GDSC_gname = GDSC2_df.columns.values

In [86]:
GDSC_gname

array(['TSPAN6', 'TNMD', 'DPM1', ..., 'POLRMTP1', 'UBL5P2', 'TBC1D3P5'],
      dtype=object)

In [87]:
CCLE_names = data.columns.values

In [88]:
CCLE_names

array(['TSPAN6 (7105)', 'TNMD (64102)', 'DPM1 (8813)', ...,
       'AL445238.1 (647264)', 'GET1-SH3BGR (106865373)',
       'AC113348.1 (102724657)'], dtype=object)

In [89]:
# Change names to upper cases
GDSC_upper_ids = []
CCLE_upper_ids = []

for gn in GDSC_gname:
    GDSC_upper_ids.append(gn.upper())
    

for gn in CCLE_names:
    name = gn.split(" (")[0]
    CCLE_upper_ids.append(name.upper())

In [90]:
CCLE_upper_ids[:5]

['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1ORF112']

In [91]:
# Get id intersect
inter_ids = (set(CCLE_upper_ids).intersection(set(GDSC_upper_ids)))

In [92]:
GDSC2_df.columns = GDSC_upper_ids
data.columns = CCLE_upper_ids

In [93]:
data.loc[:,inter_ids].head()

,MFN1,RPS18,HECW1,ASIC1,COL9A1,PAQR5,ASMT,FHOD3,CCDC116,RALGAPB,...,SLC24A2,ZCCHC13,EVI2A,SIM2,AASDHPPT,INTS3,TMEM150B,RBM27,OR10A2,DAZAP2
ACH-001113,5.969243,12.925425,2.847997,1.454176,0.000000,2.341986,0.000000,2.835924,0.084064,4.699330,...,0.201634,0.0,0.286881,0.584963,4.776630,5.104337,0.000000,3.703765,0.000000,8.548437
ACH-001289,5.130931,12.508661,0.189034,2.839960,0.000000,0.042644,0.000000,1.786596,0.150560,4.211791,...,0.042644,0.0,0.028569,0.879706,5.554282,4.641546,0.000000,3.836934,0.014355,7.896211
ACH-001339,4.638653,13.379920,0.084064,2.056584,0.000000,0.823749,0.000000,3.718088,0.275007,3.823749,...,0.275007,0.0,3.560715,0.757023,5.205549,4.450881,0.000000,3.226509,0.000000,8.418696
ACH-001538,4.772941,13.461863,0.367371,1.790772,0.014355,2.939227,0.000000,3.300124,0.097611,4.334139,...,0.000000,0.0,0.565597,1.580145,4.516646,3.709291,0.176323,4.039138,0.000000,8.044667
ACH-000242,4.527946,12.431915,0.000000,0.526069,0.014355,0.250962,0.056584,4.331275,0.014355,4.615887,...,0.014355,0.0,0.238787,3.899176,4.339137,4.813525,0.000000,3.428946,0.028569,8.265287


In [94]:
GDSC2_reordered_df = GDSC2_df.loc[:,inter_ids]

In [95]:
CCLE_reordered_df = data.loc[:,inter_ids]

In [96]:
GDSC2_reordered_df.head()

,MFN1,RPS18,HECW1,ASIC1,COL9A1,PAQR5,ASMT,FHOD3,CCDC116,RALGAPB,...,SLC24A2,ZCCHC13,EVI2A,SIM2,AASDHPPT,INTS3,TMEM150B,RBM27,OR10A2,DAZAP2
DATA.683665,8.303215,4.855941,3.003684,3.464064,3.010649,3.020780,2.839337,3.431930,3.445788,5.179192,...,3.536496,3.117370,6.783606,3.279345,8.030557,3.984573,3.229572,7.022902,3.400804,7.219612
DATA.683667,6.817876,4.680126,2.825164,4.496459,3.087131,2.793471,3.036219,5.010277,3.445690,5.178728,...,3.661130,2.948876,3.076799,3.211108,8.126359,3.677146,3.019711,6.253705,3.249729,7.344579
DATA.684052,7.426108,3.884602,2.773235,3.529227,2.958212,5.260487,2.996933,3.156271,3.053094,4.888945,...,3.215054,3.022305,3.071884,6.340367,8.929703,3.817129,3.280940,7.507780,3.118641,7.431476
DATA.684057,7.842293,4.880253,2.709583,4.208949,3.055715,5.668617,3.307322,3.612229,3.186254,5.083133,...,3.193366,3.225772,3.110857,3.995325,8.397189,4.814836,3.219291,8.628967,2.979909,7.242464
DATA.684059,7.164111,3.888214,2.816757,3.975249,3.056056,3.894122,2.810015,3.078712,3.349923,5.381896,...,3.221627,2.943814,2.762894,7.097586,8.198816,3.906015,3.604529,8.636905,3.070680,6.862803


In [97]:
CCLE_reordered_df.head()

,MFN1,RPS18,HECW1,ASIC1,COL9A1,PAQR5,ASMT,FHOD3,CCDC116,RALGAPB,...,SLC24A2,ZCCHC13,EVI2A,SIM2,AASDHPPT,INTS3,TMEM150B,RBM27,OR10A2,DAZAP2
ACH-001113,5.969243,12.925425,2.847997,1.454176,0.000000,2.341986,0.000000,2.835924,0.084064,4.699330,...,0.201634,0.0,0.286881,0.584963,4.776630,5.104337,0.000000,3.703765,0.000000,8.548437
ACH-001289,5.130931,12.508661,0.189034,2.839960,0.000000,0.042644,0.000000,1.786596,0.150560,4.211791,...,0.042644,0.0,0.028569,0.879706,5.554282,4.641546,0.000000,3.836934,0.014355,7.896211
ACH-001339,4.638653,13.379920,0.084064,2.056584,0.000000,0.823749,0.000000,3.718088,0.275007,3.823749,...,0.275007,0.0,3.560715,0.757023,5.205549,4.450881,0.000000,3.226509,0.000000,8.418696
ACH-001538,4.772941,13.461863,0.367371,1.790772,0.014355,2.939227,0.000000,3.300124,0.097611,4.334139,...,0.000000,0.0,0.565597,1.580145,4.516646,3.709291,0.176323,4.039138,0.000000,8.044667
ACH-000242,4.527946,12.431915,0.000000,0.526069,0.014355,0.250962,0.056584,4.331275,0.014355,4.615887,...,0.014355,0.0,0.238787,3.899176,4.339137,4.813525,0.000000,3.428946,0.028569,8.265287


In [98]:
intersect_CCLE_idx = CCLE_reordered_df.index.intersection(CCLE_label_common.index)
intersect_GDSC_idx = GDSC2_reordered_df.index.intersection(GDSC_label_common.index)

In [99]:
intersect_CCLE_idx

Index(['ACH-001113', 'ACH-000242', 'ACH-000461', 'ACH-000528', 'ACH-000792',
       'ACH-000570', 'ACH-000351', 'ACH-000769', 'ACH-000421', 'ACH-000765',
       ...
       'ACH-000996', 'ACH-000320', 'ACH-000505', 'ACH-000781', 'ACH-000046',
       'ACH-000968', 'ACH-000973', 'ACH-001128', 'ACH-000750', 'ACH-000052'],
      dtype='object', length=476)

In [100]:
GDSC_exp_final = GDSC2_reordered_df.loc[intersect_GDSC_idx,:]
GDSC_exp_final.head()

,MFN1,RPS18,HECW1,ASIC1,COL9A1,PAQR5,ASMT,FHOD3,CCDC116,RALGAPB,...,SLC24A2,ZCCHC13,EVI2A,SIM2,AASDHPPT,INTS3,TMEM150B,RBM27,OR10A2,DAZAP2
DATA.683665,8.303215,4.855941,3.003684,3.464064,3.010649,3.020780,2.839337,3.431930,3.445788,5.179192,...,3.536496,3.117370,6.783606,3.279345,8.030557,3.984573,3.229572,7.022902,3.400804,7.219612
DATA.683667,6.817876,4.680126,2.825164,4.496459,3.087131,2.793471,3.036219,5.010277,3.445690,5.178728,...,3.661130,2.948876,3.076799,3.211108,8.126359,3.677146,3.019711,6.253705,3.249729,7.344579
DATA.684052,7.426108,3.884602,2.773235,3.529227,2.958212,5.260487,2.996933,3.156271,3.053094,4.888945,...,3.215054,3.022305,3.071884,6.340367,8.929703,3.817129,3.280940,7.507780,3.118641,7.431476
DATA.684057,7.842293,4.880253,2.709583,4.208949,3.055715,5.668617,3.307322,3.612229,3.186254,5.083133,...,3.193366,3.225772,3.110857,3.995325,8.397189,4.814836,3.219291,8.628967,2.979909,7.242464
DATA.684059,7.164111,3.888214,2.816757,3.975249,3.056056,3.894122,2.810015,3.078712,3.349923,5.381896,...,3.221627,2.943814,2.762894,7.097586,8.198816,3.906015,3.604529,8.636905,3.070680,6.862803


In [101]:
CCLE_exp_final = CCLE_reordered_df.loc[intersect_CCLE_idx,:]
CCLE_exp_final.head()

,MFN1,RPS18,HECW1,ASIC1,COL9A1,PAQR5,ASMT,FHOD3,CCDC116,RALGAPB,...,SLC24A2,ZCCHC13,EVI2A,SIM2,AASDHPPT,INTS3,TMEM150B,RBM27,OR10A2,DAZAP2
ACH-001113,5.969243,12.925425,2.847997,1.454176,0.000000,2.341986,0.000000,2.835924,0.084064,4.699330,...,0.201634,0.0,0.286881,0.584963,4.776630,5.104337,0.000000,3.703765,0.000000,8.548437
ACH-000242,4.527946,12.431915,0.000000,0.526069,0.014355,0.250962,0.056584,4.331275,0.014355,4.615887,...,0.014355,0.0,0.238787,3.899176,4.339137,4.813525,0.000000,3.428946,0.028569,8.265287
ACH-000461,3.854993,12.072334,0.201634,1.709291,0.000000,4.346248,0.084064,0.042644,0.189034,4.311067,...,0.000000,0.0,0.084064,3.882643,3.049631,4.423578,0.678072,3.448901,0.000000,6.671010
ACH-000528,5.083639,12.337956,0.000000,4.795455,0.000000,0.411426,0.000000,0.084064,0.310340,3.715893,...,0.000000,0.0,0.097611,0.400538,5.494096,4.392317,0.000000,3.109361,0.000000,8.029619
ACH-000792,4.850999,12.058465,3.163499,3.813525,0.000000,0.855990,0.111031,3.273516,0.321928,4.259272,...,0.000000,0.0,4.883132,3.365972,3.070389,4.684819,0.000000,3.330558,0.000000,7.696203


In [102]:
CCLE_label_final = CCLE_label_common.loc[intersect_CCLE_idx,:]
CCLE_label_final.head()

,5-FLUOROURACIL,AZ960,AZD2014,AZD4547,AZD5363,AZD5438,AZD6482,AZD7762,AZD8055,BI-2536,...,TENIPOSIDE,TOPOTECAN,TOZASERTIB,TRAMETINIB,UPROSERTIB,VENETOCLAX,VINBLASTINE,VINCRISTINE,VINORELBINE,VORINOSTAT
ACH-001113,0.888576,0.765459,0.806371,1.279074,0.863918,0.833555,0.771486,0.438435,0.741458,0.412044,...,0.690835,0.580841,1.082626,0.890127,0.872673,1.219132,0.583830,0.713765,0.645499,0.706721
ACH-000242,0.873693,0.749008,0.600280,1.740781,0.897746,0.843566,0.808157,0.596800,0.536215,0.610510,...,0.640635,0.578304,0.784786,0.570094,0.756802,1.123116,0.225005,0.247308,0.706790,0.819864
ACH-000461,0.905074,0.797952,0.736532,1.752912,1.219520,0.812987,0.869369,0.538686,0.660909,0.313750,...,0.654591,0.477667,0.824032,0.763749,0.746323,0.984453,0.507130,0.684390,0.660339,0.795864
ACH-000528,0.925643,0.823479,0.780370,1.285628,0.885603,0.865272,0.740416,0.563659,0.645153,0.456717,...,0.574532,0.488042,0.696760,0.926922,0.844133,1.125290,0.465877,0.300635,0.591418,0.834818
ACH-000792,1.882904,1.219337,0.710205,NaN,0.875267,0.704461,0.783928,0.679219,0.486225,0.559383,...,0.667054,0.493800,1.000000,1.285074,0.828022,0.951512,0.379049,0.520760,0.627787,1.000000


In [103]:
GDSC_label_final = GDSC_label_common.loc[intersect_GDSC_idx,:]
GDSC_label_final.head()

,5-FLUOROURACIL,AZ960,AZD2014,AZD4547,AZD5363,AZD5438,AZD6482,AZD7762,AZD8055,BI-2536,...,TENIPOSIDE,TOPOTECAN,TOZASERTIB,TRAMETINIB,UPROSERTIB,VENETOCLAX,VINBLASTINE,VINCRISTINE,VINORELBINE,VORINOSTAT
DATA.683665,0.988746,0.670322,0.950175,0.968525,0.961817,0.764785,NaN,0.492478,NaN,NaN,...,0.427355,0.618331,NaN,0.994088,0.936213,0.933957,0.483393,0.072019,0.645842,0.744778
DATA.683667,0.976741,0.860038,0.847139,0.994262,0.876508,0.812361,NaN,0.648063,NaN,NaN,...,0.851304,0.861507,NaN,0.989145,0.848736,0.887319,0.599965,0.291710,0.710986,0.695245
DATA.684052,0.949240,0.807361,0.778575,0.948738,0.935697,0.713411,NaN,0.589910,NaN,NaN,...,0.574182,0.466748,NaN,0.965742,0.953161,0.971618,0.460939,0.174685,0.640397,0.800694
DATA.684057,0.959395,0.730258,0.674690,NaN,NaN,0.844561,NaN,NaN,NaN,NaN,...,0.747489,0.834007,NaN,NaN,0.777959,NaN,NaN,0.240994,NaN,NaN
DATA.684059,0.858744,0.756592,0.750252,0.982099,0.934719,0.761498,NaN,0.475531,NaN,NaN,...,0.467434,0.312764,NaN,0.989165,0.920961,0.964455,0.498375,0.106963,0.529711,0.785805


In [127]:
CCLE_all = dose_tab.loc[intersect_CCLE_idx,:]
GDSC_all = gdsc_tab.loc[intersect_GDSC_idx,:]

In [153]:
CCLE_all.head()

,1-AZAKENPAULLONE,1-NAPHTHYL-PP1,1-PHENYLBIGUANIDE,10-DEACETYLBACCATIN,10-HYDROXYCAMPTOTHECIN,12-O-TETRADECANOYLPHORBOL-13-ACETATE,2'-MECCPA,"2,3-DCPE",2-METHOXYESTRADIOL,3'-FLUOROBENZYLSPIPERONE,...,WARFARIN,XANOMELINE,XILOBAM,XYLAZINE,ZALDARIDE,ZALEPLON,ZARDAVERINE,ZIPRASIDONE,ZOLANTIDINE,ZOLMITRIPTAN
ACH-001113,0.914946,0.800483,0.955256,0.863754,0.287046,0.877779,2.360874,1.056416,0.727263,1.491698,...,0.887100,0.829147,1.376536,1.705097,1.367165,1.319688,0.849210,1.967298,0.903250,1.662385
ACH-000242,1.527743,1.136690,0.873219,0.803828,0.303859,0.688172,1.512568,1.058299,0.812375,1.281235,...,1.771727,0.781597,0.863747,1.778585,1.452808,NaN,0.989214,1.528785,0.943743,0.853210
ACH-000461,0.858458,0.783264,0.888583,0.831949,0.285621,0.787543,1.316506,0.947339,0.877231,1.326461,...,1.188109,0.834334,1.304282,1.591300,1.351373,0.798556,0.828687,0.970269,NaN,0.905065
ACH-000528,1.258848,0.891512,0.872195,0.843942,0.278302,NaN,1.411280,1.046897,0.909233,0.988230,...,0.852684,0.863241,0.881528,1.293671,0.923692,0.966609,0.750108,0.928064,0.938372,0.868230
ACH-000792,1.750478,1.367146,1.608174,0.880541,0.272516,0.877581,1.668328,1.463802,0.976455,1.134739,...,1.000000,1.426040,NaN,1.603929,1.232021,1.000000,0.726195,1.243366,NaN,1.186371


In [145]:
GDSC_CCLE_label_all = pd.concat([GDSC_all,CCLE_all])

In [149]:
GDSC_label_final.to_csv("data/GDSC2_label_final.csv")
CCLE_label_final.to_csv("data/CCLE_label_final.csv")
GDSC_exp_final.to_csv("data/GDSC2_expression_final.csv")
CCLE_exp_final.to_csv("data/CCLE_expression_final.csv")
CCLE_all.to_csv("data/CCLE_label_all.csv")
GDSC_all.to_csv("data/GDSC_label_all.csv")

In [142]:
drugs_testinf = ["cisplatin","gefitinib","erlotinib","I-BET-762","Docetaxel",\
                 "Afatinib","AZD5591","Everolimus","Gemcitabine","JQ1","Prexasertib",\
                "Taselisib","Trametinib"]

In [140]:
CCLE_label_final[sd].count()

474

In [152]:
for d in drugs_testinf:
    sd = d.upper()
    try:
        count = CCLE_all[sd].count()

        print(sd+ " have " + str(count))
    except:
        print(sd+" NOt found")

CISPLATIN have 443
GEFITINIB have 463
ERLOTINIB have 474
I-BET-762 have 467
DOCETAXEL have 474
AFATINIB have 467
AZD5591 NOt found
EVEROLIMUS have 459
GEMCITABINE have 384
JQ1 NOt found
PREXASERTIB NOt found
TASELISIB have 462
TRAMETINIB have 474


In [151]:
for d in drugs_testinf:
    sd = d.upper()
    try:
        count = GDSC_all[sd].count()

        print(sd+ " have " + str(count))
    except:
        print(sd+" NOt found")

CISPLATIN have 765
GEFITINIB have 749
ERLOTINIB have 749
I-BET-762 have 725
DOCETAXEL have 799
AFATINIB have 801
AZD5591 NOt found
EVEROLIMUS NOt found
GEMCITABINE have 753
JQ1 have 47
PREXASERTIB NOt found
TASELISIB have 800
TRAMETINIB have 767


In [146]:

for d in drugs_testinf:
    sd = d.upper()
    try:
        count = GDSC_CCLE_label_all[sd].count()

        print(sd+ " have " + str(count))
    except:
        print(sd+" NOt found")

CISPLATIN have 1208
GEFITINIB have 1212
ERLOTINIB have 1223
I-BET-762 have 1192
DOCETAXEL have 1273
AFATINIB have 1268
AZD5591 NOt found
EVEROLIMUS have 459
GEMCITABINE have 1137
JQ1 have 47
PREXASERTIB NOt found
TASELISIB have 1262
TRAMETINIB have 1241


In [158]:
res_idx = CCLE_label_final>0.8
sens_idx = CCLE_label_final<=0.8

CCLE_label_final[res_idx]="resistant"
CCLE_label_final[sens_idx]="sensitive"

In [163]:
CCLE_label_final.head()

,5-FLUOROURACIL,AZ960,AZD2014,AZD4547,AZD5363,AZD5438,AZD6482,AZD7762,AZD8055,BI-2536,...,TENIPOSIDE,TOPOTECAN,TOZASERTIB,TRAMETINIB,UPROSERTIB,VENETOCLAX,VINBLASTINE,VINCRISTINE,VINORELBINE,VORINOSTAT
ACH-001113,resistant,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,sensitive,sensitive,...,sensitive,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,sensitive,sensitive
ACH-000242,resistant,sensitive,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,sensitive,...,sensitive,sensitive,sensitive,sensitive,sensitive,resistant,sensitive,sensitive,sensitive,resistant
ACH-000461,resistant,sensitive,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,sensitive,...,sensitive,sensitive,resistant,sensitive,sensitive,resistant,sensitive,sensitive,sensitive,sensitive
ACH-000528,resistant,resistant,sensitive,resistant,resistant,resistant,sensitive,sensitive,sensitive,sensitive,...,sensitive,sensitive,sensitive,resistant,resistant,resistant,sensitive,sensitive,sensitive,resistant
ACH-000792,resistant,resistant,sensitive,NaN,resistant,sensitive,sensitive,sensitive,sensitive,sensitive,...,sensitive,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,sensitive,resistant


In [164]:
CCLE_label_final.to_csv("data/CCLE_label_binary_08.csv")

In [165]:
res_idx = GDSC_label_final>0.8
sens_idx = GDSC_label_final<=0.8

GDSC_label_final[res_idx]="resistant"
GDSC_label_final[sens_idx]="sensitive"

In [168]:
GDSC_label_final.to_csv("data/GDSC_label_binary_08.csv")

In [176]:
CONC_label_df = pd.concat([GDSC_label_final,CCLE_label_final])
CONC_label_df.to_csv("data/ALL_label_binary_08.csv")
CONC_label_df

,5-FLUOROURACIL,AZ960,AZD2014,AZD4547,AZD5363,AZD5438,AZD6482,AZD7762,AZD8055,BI-2536,...,TENIPOSIDE,TOPOTECAN,TOZASERTIB,TRAMETINIB,UPROSERTIB,VENETOCLAX,VINBLASTINE,VINCRISTINE,VINORELBINE,VORINOSTAT
DATA.683665,resistant,sensitive,resistant,resistant,resistant,sensitive,NaN,sensitive,NaN,NaN,...,sensitive,sensitive,NaN,resistant,resistant,resistant,sensitive,sensitive,sensitive,sensitive
DATA.683667,resistant,resistant,resistant,resistant,resistant,resistant,NaN,sensitive,NaN,NaN,...,resistant,resistant,NaN,resistant,resistant,resistant,sensitive,sensitive,sensitive,sensitive
DATA.684052,resistant,resistant,sensitive,resistant,resistant,sensitive,NaN,sensitive,NaN,NaN,...,sensitive,sensitive,NaN,resistant,resistant,resistant,sensitive,sensitive,sensitive,resistant
DATA.684057,resistant,sensitive,sensitive,NaN,NaN,resistant,NaN,NaN,NaN,NaN,...,sensitive,resistant,NaN,NaN,sensitive,NaN,NaN,sensitive,NaN,NaN
DATA.684059,resistant,sensitive,sensitive,resistant,resistant,sensitive,NaN,sensitive,NaN,NaN,...,sensitive,sensitive,NaN,resistant,resistant,resistant,sensitive,sensitive,sensitive,sensitive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000968,resistant,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,sensitive,sensitive,...,sensitive,sensitive,resistant,sensitive,resistant,resistant,sensitive,sensitive,sensitive,sensitive
ACH-000973,resistant,sensitive,resistant,resistant,resistant,sensitive,sensitive,sensitive,sensitive,sensitive,...,sensitive,sensitive,sensitive,resistant,resistant,resistant,sensitive,sensitive,sensitive,resistant
ACH-001128,resistant,sensitive,sensitive,NaN,NaN,NaN,sensitive,sensitive,NaN,NaN,...,sensitive,sensitive,sensitive,sensitive,NaN,resistant,sensitive,NaN,sensitive,NaN
ACH-000750,resistant,resistant,sensitive,resistant,resistant,NaN,sensitive,sensitive,NaN,sensitive,...,sensitive,sensitive,sensitive,sensitive,resistant,resistant,sensitive,sensitive,sensitive,sensitive


In [177]:
CONC_exp_df = pd.concat([GDSC_exp_final,CCLE_exp_final])
CONC_exp_df.to_csv("data/ALL_expression.csv")
CONC_exp_df

,ST3GAL2,NOTCH2,GRPEL1,SHC2,MAGEA8,GNA14,KRTCAP2,CEP44,EHD2,UTP14C,...,CIAPIN1,PPFIA4,EPPK1,PSPC1,SLC30A2,RNF13,LAMC1,PITX3,SNX24,IL17RE
DATA.683665,5.144679,3.223748,8.473993,3.367309,3.228205,2.980778,8.523072,3.568470,3.205451,6.173123,...,5.794856,3.115977,3.236775,4.614050,3.334627,8.512252,5.454001,3.579842,3.719559,3.208217
DATA.683667,4.891309,4.978066,8.965003,7.597621,2.939119,3.127610,9.164316,3.226157,3.594308,4.838035,...,5.429998,3.165675,3.293903,3.833966,3.115850,9.187912,7.297341,3.350470,6.678025,3.106149
DATA.684052,4.185672,4.580091,8.342951,3.454799,2.940054,3.842441,8.715775,3.509050,2.943156,5.688459,...,6.166544,3.070921,4.455511,4.108411,3.080420,7.986220,7.260711,3.123496,4.627073,3.088657
DATA.684057,4.103474,4.681022,7.894892,6.939582,3.098473,4.450950,9.803396,3.732709,2.991773,6.523652,...,5.617250,3.318546,8.393024,3.945185,3.074119,8.615635,6.793222,3.177603,4.703124,3.081756
DATA.684059,4.564867,4.076783,8.136921,7.620565,3.111922,6.328582,8.801057,3.633442,3.264081,6.073701,...,5.948226,3.203330,3.278578,3.672100,3.171120,8.673449,6.673377,3.496695,5.829292,3.210553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000968,3.536053,5.129695,5.240314,0.286881,0.028569,0.014355,7.193082,2.235727,4.712596,2.575312,...,5.188638,0.956057,0.014355,5.753016,0.176323,5.068241,6.178914,0.028569,4.613532,2.482848
ACH-000973,3.682573,4.264536,6.050502,0.014355,0.084064,0.000000,6.989253,2.669027,3.684819,2.914565,...,5.704319,2.117695,0.443607,6.460415,0.014355,3.642702,4.507795,0.084064,3.685940,0.464668
ACH-001128,5.620000,3.434295,5.324451,1.678072,0.000000,0.275007,7.228049,4.339850,5.921484,3.295723,...,6.110405,0.903038,0.014355,5.503349,0.176323,4.854993,5.902315,0.584963,4.188243,3.767655
ACH-000750,3.276497,5.141188,5.980025,0.042644,0.000000,0.056584,7.113117,3.431623,4.481557,2.289834,...,5.069960,0.411426,0.014355,5.667608,0.000000,4.963012,5.705978,0.000000,4.381975,0.575312


In [179]:
CCLE_label_waterfall = pd.read_csv("data/CCLE_label_all_drugs_binary.csv",index_col=0)
CCLE_label_waterfall

,X5.FLUOROURACIL,AZ960,AZD2014,AZD4547,AZD5363,AZD5438,AZD6482,AZD7762,AZD8055,BI.2536,...,TENIPOSIDE,TOPOTECAN,TOZASERTIB,TRAMETINIB,UPROSERTIB,VENETOCLAX,VINBLASTINE,VINCRISTINE,VINORELBINE,VORINOSTAT
ACH-001113,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,...,sensitive,sensitive,resistant,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive
ACH-000242,sensitive,sensitive,sensitive,resistant,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,...,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive
ACH-000461,sensitive,sensitive,sensitive,resistant,resistant,sensitive,sensitive,sensitive,sensitive,sensitive,...,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive
ACH-000528,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,...,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive
ACH-000792,resistant,resistant,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,...,sensitive,sensitive,resistant,resistant,sensitive,sensitive,sensitive,sensitive,sensitive,resistant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000968,resistant,sensitive,sensitive,sensitive,resistant,sensitive,sensitive,sensitive,resistant,sensitive,...,sensitive,resistant,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive
ACH-000973,sensitive,sensitive,sensitive,sensitive,resistant,sensitive,sensitive,resistant,sensitive,sensitive,...,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive
ACH-001128,sensitive,sensitive,sensitive,sensitive,resistant,resistant,sensitive,sensitive,resistant,resistant,...,sensitive,sensitive,sensitive,sensitive,resistant,sensitive,sensitive,resistant,sensitive,resistant
ACH-000750,resistant,sensitive,sensitive,sensitive,sensitive,resistant,sensitive,sensitive,resistant,sensitive,...,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive


In [180]:
GDSC_label_waterfall = pd.read_csv("data/GDSC2_label_all_drugs_binary.csv",index_col=0)
GDSC_label_waterfall

,X5.FLUOROURACIL,AZ960,AZD2014,AZD4547,AZD5363,AZD5438,AZD6482,AZD7762,AZD8055,BI.2536,...,TENIPOSIDE,TOPOTECAN,TOZASERTIB,TRAMETINIB,UPROSERTIB,VENETOCLAX,VINBLASTINE,VINCRISTINE,VINORELBINE,VORINOSTAT
DATA.683665,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,...,sensitive,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,resistant,resistant
DATA.683667,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,...,resistant,resistant,resistant,resistant,resistant,sensitive,sensitive,sensitive,resistant,resistant
DATA.684052,resistant,resistant,resistant,resistant,resistant,sensitive,resistant,resistant,resistant,resistant,...,sensitive,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,resistant,resistant
DATA.684057,resistant,resistant,sensitive,resistant,resistant,resistant,resistant,resistant,resistant,resistant,...,resistant,resistant,resistant,resistant,resistant,resistant,resistant,sensitive,resistant,resistant
DATA.684059,resistant,resistant,sensitive,resistant,resistant,resistant,resistant,resistant,resistant,resistant,...,sensitive,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,sensitive,resistant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DATA.1659928,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,sensitive,resistant,...,resistant,resistant,resistant,resistant,resistant,resistant,sensitive,sensitive,sensitive,resistant
DATA.1660034,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,sensitive,resistant,...,resistant,resistant,resistant,sensitive,resistant,resistant,sensitive,resistant,resistant,resistant
DATA.1660035,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,...,resistant,resistant,resistant,sensitive,resistant,resistant,resistant,resistant,resistant,resistant
DATA.1660036,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,sensitive,resistant,...,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant


In [181]:
CONC_wf_df = pd.concat([GDSC_label_waterfall,CCLE_label_waterfall])
CONC_wf_df.to_csv("data/ALL_label_binary_wf.csv")
CONC_wf_df

,X5.FLUOROURACIL,AZ960,AZD2014,AZD4547,AZD5363,AZD5438,AZD6482,AZD7762,AZD8055,BI.2536,...,TENIPOSIDE,TOPOTECAN,TOZASERTIB,TRAMETINIB,UPROSERTIB,VENETOCLAX,VINBLASTINE,VINCRISTINE,VINORELBINE,VORINOSTAT
DATA.683665,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,...,sensitive,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,resistant,resistant
DATA.683667,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,resistant,...,resistant,resistant,resistant,resistant,resistant,sensitive,sensitive,sensitive,resistant,resistant
DATA.684052,resistant,resistant,resistant,resistant,resistant,sensitive,resistant,resistant,resistant,resistant,...,sensitive,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,resistant,resistant
DATA.684057,resistant,resistant,sensitive,resistant,resistant,resistant,resistant,resistant,resistant,resistant,...,resistant,resistant,resistant,resistant,resistant,resistant,resistant,sensitive,resistant,resistant
DATA.684059,resistant,resistant,sensitive,resistant,resistant,resistant,resistant,resistant,resistant,resistant,...,sensitive,sensitive,resistant,resistant,resistant,resistant,sensitive,sensitive,sensitive,resistant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000968,resistant,sensitive,sensitive,sensitive,resistant,sensitive,sensitive,sensitive,resistant,sensitive,...,sensitive,resistant,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive
ACH-000973,sensitive,sensitive,sensitive,sensitive,resistant,sensitive,sensitive,resistant,sensitive,sensitive,...,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive
ACH-001128,sensitive,sensitive,sensitive,sensitive,resistant,resistant,sensitive,sensitive,resistant,resistant,...,sensitive,sensitive,sensitive,sensitive,resistant,sensitive,sensitive,resistant,sensitive,resistant
ACH-000750,resistant,sensitive,sensitive,sensitive,sensitive,resistant,sensitive,sensitive,resistant,sensitive,...,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive,sensitive
